# Document-Oriented Data Modeling

*Last updated: September 30, 2021*

This notebook shares how Aerospike facilitates working with document-oriented data, covering the following topics:

1. Finding the balance between size and application performance
2. Tradeoffs of denormalized and normalized data modeling
3. Modeling 1-to-many relationships
4. Modeling 1-to-1 relationships
5. Modeling complex relationships

The above topics showcase the dos and donts of modeling in Aerospike and combine both product features and realities of data modeling into a collection of best practices.

This [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html) requires the Aerospike Database running locally with Java kernel and Aerospike Java Client. To create a Docker container that satisfies the requirements and holds a copy of these notebooks, visit the [Aerospike Notebooks Repo](https://github.com/aerospike-examples/interactive-notebooks).

# Prerequisites
* [Intro to Data Modeling](./java-intro_to_data_modeling.ipynb)
* [Modeling Using Lists](./java-modeling_using_lists.ipynb)
* [Modeling Using Maps](./java-modeling_using_maps.ipynb)


# Whether to normalize or denormalize the data...

The grand question when working with semi-structured or Document-Oriented Data is when to efficiently store just one copy of data (Normalize the data) or store more copies of data as required by the use cases as subdocuments (Denormalize). TL:DR, it’s a balance. The ultimate goal of this balancing act is to ensure that your Aerospike cluster is responsive to more requests, and is not tied up in excessively I/O-, CPU-, or RAM/Storage-intensive operations. Aerospike cluster configurations afford significant flexibility, but ultimately the best practice is to create data for which: 

* Reads – the application has sufficiently fast access to the data needed for the app
* Writes – having space and availability to write app data without running into Aerospike [known limitations](https://docs.aerospike.com/docs/guide/limitations.html).

Ultimately this depends on what data is needed for efficient transaction processing using Aerospike’s record-atomic transactions using the operate API method. By the nature of applications, the higher frequency of a type of transaction, the stronger the need for atomicity. 


# Finding the Balance between Size and Application Performance 

Simply put, larger records means larger reads from storage. In turn, larger reads from storage generally means a database will take at least as much time to access a record of data as a smaller record. At some point that is different for every database, application, and record type, too large of a record results in inadequate application performance. 

Aerospike has a large sweet spot for real time performance and Aerospike records technically have no size limit if stored in memory, starting with version 4.5.3.4. However, memory is finite, and therefore most records are written to a storage device. The most common range of record sizes for real time performance at scale is from 1 Byte to 128 KBs. At that size and using Aerospike default configuration:
Performance times are more directly related to storage medium than processor or transport. (This is my understanding. Please vet if this is a safe claim to make.) 
Due to the current speed of network I/O, smaller sized records are only marginally faster.

**Note:** Customers routinely create and use records that are larger than the 128-KB range. The maximum size for data stored on a device is 8 MiB. 

Best practices are:
1. Work with the Aerospike team to help you tune for performance for MB-sized records or extreme read/write workloads. You may also seek advice on Aerospike’s [Tuning Forum](https://discuss.aerospike.com/c/operations/tuning/23). 
2. Verify the server and client versions support necessary features.
3. Test Aerospike cluster settings in development prior to production deployment to ensure hardware and software behave as-expected under the new configuration.

## Types of Reads in Aerospike

The techniques to read a record are:
* Record read – A 1-record read.
* Batch record read – A multiple record read.
* Scan, a primary index read of all records in a Namespace
* Secondary index query – A scan of a subset of records, best used when the subset is 1/100 of the overall namespace. Must be explicitly created. 

The consequence of not embedding appropriate references is steeper than an RDBMS.
Best practices for reads are therefore to:
* Embed references to related records in models. Create secondary indexes. 
* Embed Aerospike expressions into reads to select data from within a record. 



# Tradeoffs of Denormalized/Normalized Data Modeling

The following are inherent tradeoffs made by denormalizing or normalizing data in an app’s data model.


## Trade Offs of Denormalization

Denormalizing data is a tradeoff where the application writes data in multiple places at creation/modification time for more efficient transaction processing. 

Nesting data as a subdocument helps to reduce drive I/O, CPU and RAM overhead. The subdocument can be part of a single-record transaction using the operate method. If some of the data for a transaction were located in a separate record, that would mandate additional complexity resulting from the loss of atomicity and the addition of multiple record locks and reads of data from media. For efficiency, Aerospike serializes Map data using [Message Pack](https://msgpack.org/index.html). 


## Trade Offs of Normalization

Normalizing data adds complexity to data queries and updates, and it sacrifices some performance to improve storage efficiency.  The main reasons to normalize data and incur the additional transactional complexity are:

1. To make efficient use of storage media. 
2. Store more important or more frequently-used data on faster media. 
3. Prevent relatively unused data from slowing down transaction performance through bloating the record.  

Normalize data that is more static (fewer writes) and doesn’t require high volumes of reads. 


The following are situations and examples that suggest when to denormalize and normalize data. 


## Denormalization for Performance
Store data together if it is read together. For example, storing a company’s address in the same record as company name:

```
company: {
	"name": “Aerospike, Inc",
	"address”: {
			"street": "2525 E Charleston Road"
			"city": “Mountain View"
			"state": “CA"
			"zip": "94043"
		}
}
```

Despite being a many to many relationship, companies can have multiple addresses, and each address can be used by multiple companies, this information is regularly used together.


## Denormalization for Space Considerations

Consider the task of modeling an application with millions or billions of tiny (~80-byte) objects when the chief concern is efficient storage and not performance. Since each Aerospike record has 64-byte of overhead in the index, by architecting a data model consisting of many tiny application objects each as its own record, the architect would be manufacturing a capacity concern. Instead, try the following best practices:

* Modeling: Consolidate into Maps
* Aerospike configuration: Use All-Flash


### Consolidate into A Map with Object IDs as Mapkeys

Consolidate objects into one map with object IDs as mapkeys to reduce I/O and index memory. The 64-byte overhead in the index is small relative to 600-byte or greater record size. Potentially group these mapkeys in records whose key pattern should be modeled after time duration (hour or day). 


### Use All-Flash

When an Aerospike Namespace is configured to use All-Flash, Aerospike stores both data and index on SSD. This configuration consumes little memory, significantly less than when the index is stored in memory. The trade offs are minor: 
Aerospike using All-Flash costs an extra read from SSD to access the record’s metadata from the index.
Writes have higher I/O cost than other configurations, because index is written to a slower medium. 


# Modeling 1:Many Relationships

When modeling a 1:many relationship, common techniques are:
* Denormalization: Embedding a Document Pattern inside another document. 
* Normalization: Embedding explicit reference to other records.


## Denormalized 1:Many Modeling: Embedding a Document Pattern

This is appropriate when data records are fully contained in another record, for example, a standard 1:many relationship, such as one user with multiple credit cards, as modeled in [Modeling Using Maps](./java-modeling_using_maps.ipynb) as a list of credit card Maps. 

To support this model, it can be necessary to access the embedded document directly. If the model needs to access cards independently of the user-record where it is embedded, here are common techniques:
1. Create a universally unique ID (UUID) for each card:
   * Hash immutable card details into a new UUID
   * Use an existing app unique ID
2. Create a way to lookup cards within the user record by UUID:
   * Create a bin storing a List of UUIDs in the record 
   * Manage your own index of UUIDs.

To access a card, create a secondary index on the list, or scan the index using [expressions](https://docs.aerospike.com/docs/guide/expressions/). A secondary index is faster, but consumes more storage media. Expressions are slower than scanning a secondary index and use more CPU overhead.

### Creating an Index to Access Embedded Elements 

**Example:** The application needs to check if a credit card appears in more than one user account. 

```
user: {
	cards: [
		{
			“cvv”: 111
			“expires”: 202201
			“last_six”: 511111
			“zip”: 95008
		}
		{
			“cvv”: 222
			“expires”: 202202
			“last_six”: 522222
			“zip”: 95008
		}
		{
			“cvv”: 333
			“expires”: 202203
			“last_six”: 533333
			“zip”: 95008
		}
		]
	}
cardhashes: [
		“e993d2dc-b060-46e2-a9e2-73b7dfb63dd0”
		“0fbcb8eb-8e22-47b6-ba76-b6ae2f2d00ae”
		“d5d68ddf-6476-497a-af92-266b32072905”
		]
```

**Model:**
* Carduser is an in-memory Namespace with a 1:1 relationship to records in the user set. 
* Key: A canonical form of the card data: 
```
CVV | Expires | Last 6 Digits | Zip – “219|202210|518923|95008”
```
* Value: The digest of the user record.

**Operations:**
* Create – Use write policy CREATE_ONLY to create this after successful card write to a user record.
* Exists – A metadata-only check. If the key exists, it exists.
* Read – Returns the user record digest for subsequent fetching.
* Delete – TTL set to card expiry, or explicitly by durable deletes.

**Advantage:** A single read is faster than scanning all records in an index.

## Normalized 1:Many Modeling: Embedding Explicit References
When building a normalized one to many model, it is necessary to embed identifiers for related records. Such a relationship cannot be implied, as will be modeled discussing 1:1 relationships, below.

### Creating Embedded References to other Records 

**Example:** Modeling a user and his/her mobile devices, where user details need to be accessed less frequently than mobile device details 

```
Record: (ns1-ssd, user, uid1991)
	{
		“devices”: [
“ZSKF3B9HCLKJ”
]
		“address”: {
				“zip”: 94043
			}
		“first_name”: “Thomas”
		“last_name”: “Anderson”
	}


Record: (ns1-ssd, device, ZSKF3B9HCLKJ)
	{
		“name”: “Donkey Phone”
		“model_name”: “iPhone X”
		“model_num”: “MQQKK2NN/B”
		“serial_num”: “ZSKF3B9HCLKJ”
		“user”: “uid1991”
	}
```

**Model:**
* Namespace: All data is stored on ssd.
* Sets: Application objects user and device are stored as sets of records. 
* Keys: 
   * Application UserID is used to uniquely identify a user. 
   * Device Serial Number is used to uniquely identify a device.  
* References
   * Device.user identifies the user who owns the device.
   * User.devices contains the list of devices owned by the user.

**Advantage:** Device updates do not need to access the user record, which largely contains identifying information. 

# Modeling 1:1 Relationships
Denormalized modeling of 1:1 relationships is embedding objects into one another. Normalized modeling of 1:1 relationships uses separate objects instead of embedding one in another.

## Implied References: Modeling a 1:1 Relationship By Varying One: Namespace/Set/Key
As described in the [Introduction to Data Modeling tutorial](./java-intro_to_data_modeling.ipynb), the Aerospike data model elements Namespace, Set, and Key, together, uniquely identify a record. Each provides specifies a type of control over data –
* Namespace – Server-configured link between data and hardware.
* Set – Optional classification to easily manage multiple records of data together. 
* Key – A user-defined name for a record of data.

A powerful way to leverage Aerospike’s architecture is to align a 1:1 relationship between two Aerospike records by aligning two Record identifiers and changing the third. For example, storing one record in one Namespace and another in another Namespace, while using the same Key and Set name. This is similar to using an [Implied Foreign Key](https://docs.oracle.com/en/database/oracle/sql-developer-web/19.1/sdweb/implied-foreign-keys-dialog.html) configuration in a relational database. 

### Examples of Implied References
Common examples where implied references are used are as follows:
* Fragmenting the record by frequency of data usage.
* Accommodating dimensions of data growth or rotation by discrete ranges.


#### Fragmenting by Frequency of Usage
When an application record contains data that is read, written, or deleted at significantly different frequencies, a common technique is to place the frequently used data in one Aerospike Record, and the rest in a second Record accessible through implied reference.


#### Fragmenting a Record and Accessing by Implied Reference

**Example:** Tracking data consumption for a mobile phone.

```
record: (ns1-ssd, device, ZSKF38B9HCLKJ)

device: 
	{
	“name”: “Donkey Phone”
	“model_name”: “iPhone X”
	“model_num”: “QQKK2NN/B”
	“serial_num”: “ZSKF3B9HCLKJ”
	“user”: “uid1991”
}

record: (ns1-memory, device, ZSKF38B9HCLKJ)

bytes: 6442450216
```

**Model:**
* Most phone information is static. Store it on SSD.
* Data consumption in bytes (an 8-byte integer) changes frequently. Store it in-memory.
* Set: ‘device’, distinguishing device data from user data, etc.. 
* Key: ‘ZSKF38B9HCLKJ’, the serial number of the device.
* Namespaces:
   * ‘ns1-ssd’: a Namespace storing data on SSD media.
   * ‘ns1-memory’: a Namespace storing data in memory.  

**Advantage:** Because the Set and Key are identical, the records share the same Record Digest, though being stored on different media. This configuration allows for intuitive access from the application, and results in much lower overhead than reading the full record, incrementing, and writing the full record.

#### Storing Data by Discrete Data Range
For data that grows in discrete ranges, add the data to discrete records in the same Set. 

**Example:** Storing IoT sensor data by date.

```
Record: (ns1-ssd, sensor, 1|9876) 
[ 
  [1, 2345]
  [2, 2415]
  [3, 2570]
...
  [1440, 2313]
]

Record: (ns1-ssd, sensor, 2|9876) 
[ 
  [1, 2419]
  [2, 2512]
  [3, 2590]
...
  [1440, 2402]
]

Record: (ns1-ssd, sensor, 99|9876) 
[ 
  [1, 3176]
  [2, 3241]
  [3, 3371]
...
  [1440, 3213]
]

Record: (ns1-ssd, sensor, 100|9876)        Today. 
[ 
  [1, 3205]
  [2, 3295]
  [3, 3361]
]
```


**Model:**
* Sensor data is written every minute for a year, as a list of tuples `[epoch-minute, temperature]`. Records are written one day, then remain immutable.
* Key: `day-since-epoch|sensor-id`

**Operations:**
* Create – Use append to add new sensor readings.
* Read – Retrieve data using batch reads over the date key ranges.
* Delete – TTL set to data rotation period, or explicitly by durable deletes.

**Advantage:** Clean data access and rotation. No need to index, because implied references indicate data location. Getting a year’s worth of data is just a batch read of 365 records. Multiple sensors for a given day is just as easy. Batch reads are tolerant of missing keys.

# Modeling Complex Relationships

The more complex the relationships, the more likely embedding references to other records is constructive.


## Multiple embedded references to records  

**Example:** Reddit posts and karma

```
User Record: (ns1-ssd, user, donkongster)
user:
{
“name”: “Don Kekong”,
“since”: 20181012,
“karma”: 72,
“trophies”: {“t1”, “t8”, “t19”},
“posts”: {“17-891”, “38-767”, “51-209”},    		// Reference to UUID of Post
“comments”: [“13-191”, “98-209”, :”77-101”},	// Ref to UUID of Comment 
“upvotes”: [“21-876”, “90-898”},
“downvotes”: {‘81-023”}
}

Comment Record: (ns1-ssd, comments, 90-209)
comment:
{
“txt”: “🤣🤣 *”,
“post”: “17-891”,						// Ref to UUID of Post
“user”: “donkongster”,					// Ref to UUID of User
“posted”: 288034,
“votes”: 5
}

Post Record: (ns1-ssd, post, 17-891)
post:
{
“title”: “Hilarity*”,
“posted”: 288000,
“uri”: “https://goo.gl/foo012”,
“user”: “donkongster”,					// Ref to UUID of User
“votes”: 17,
“comment_count”: 3,
“comments”: [“60-993”, “98-209”, “55-123”, “21-235”],	// Ref to UUID of Comment
“comment_tree: [
[“60-993, [“98-209”, “55-123”], “21-235”]
],
“body”: “”
}
```

**Model**:
* Posts to Comments: 1:many
* Users to Comments: 1:many
* Users to Posts: 1:many
* Assume no complex joins and separate HTTP requests per page, which can resolve independently.

**Operations**:
   * Page is a batch read loading post and comments
      * A Post Page can be loaded simply:
         * Load the Post record
         * Batch read the comments records 
      * A Comments Page is a batch read of comments UUIDs. Lists support pagination.
      * A User’s Posts Page is a batch read of all of the post UUIDs from the User record.
   * Upvote/downvote a comment is two updates:
   * Update the karma and upvote or downvote in the User record.
   * Increment the vote counter in the Comment record. 
   * New posts, comments, etc. would be prepended and not appended, so newest content is first.

**Advantage**: Relevant content references are always available, when needed. Primary content for page loads (e.g. a post) will load faster than secondary content (e.g., comments). All content loads within visitor expectations.

# Takeaways

Aerospike Database empowers both normalized and denormalized modeling use cases common to working with document-oriented data models. 

## What's Next?

### Next Steps

Have questions? Don't hesitate to post about modeling using maps on [Aerospike's Discussion Forums](https://discuss.aerospike.com/c/how-developers-are-using-aerospike/data-modeling/143).

Want to check out other Java notebooks?
1. [Hello, World](./hello_world.ipynb)
2. [Aerospike Query and UDF](./query_udf.ipynb)
3. [Simple Put Get Example](./SimplePutGetExample.ipynb)
4. [Expressions](./expressions.ipynb)
5. [Advanced Collection Data Types](./java-advanced_collection_data_types.ipynb)

Are you running this from Binder? [Download the Aerospike Notebook Repo](https://github.com/aerospike-examples/interactive-notebooks) and work with Aerospike Database and Jupyter locally using a Docker container.

## Additional Resources

* Want to get started building with Aerospike? [Try now](https://aerospike.com/lp/try-now/).
* Ready to learn about developing with Aerospike? Go to the [Developer Hub](https://developer.aerospike.com).
* How robust is the Aerospike Database? Browse the [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html).
* Want to get started with Java? [Download](https://www.aerospike.com/download/client/) or [install](https://github.com/aerospike/aerospike-client-java) the Aerospike Java Client. 
* How robust is the Aerospike Database? Browse the [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html).